In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm


# Pour le preproccess
import spacy as sp
nlp = sp.load('en_core_web_sm')

In [ ]:
# On a décidé de ne prendre que 15 sites (cf. Stats_Desc_Nature_V0.ipynb):

list_chosen_site = ['Scientific Reports',
                    'Nature Communications',
                    'Cell Death & Disease',
                    'Oncogene',
                    'Nature',
                    'Neuropsychopharmacology',
                    'British Journal of Cancer',
                    'Leukemia',
                    'Acta Pharmacologica Sinica',
                    'Nature Medicine',
                    'Translational Psychiatry',
                    'Nature Neuroscience',
                    'Molecular Psychiatry',
                    'Nature Protocols',
                    'Cell Death & Differentiation']

In [ ]:
#Chargement des CSV en un (à opti)
df_all = pd.concat([pd.read_csv("../Data/Nature/articles_Nature_data_1_V0.csv", sep=";"),
                    pd.read_csv("../Data/Nature/articles_Nature_data_2_V0.csv", sep=";"),
                    pd.read_csv("../Data/Nature/articles_Nature_data_3_V0.csv", sep=";"),
                    pd.read_csv("../Data/Nature/articles_Nature_data_fin_V0.csv", sep=";")])


#Suppression de la colonne doi et de l'indice des articles qui est en double
df_all = df_all.drop(df_all.columns[[0, 1]], axis='columns')


Si trop lourd pour la RAM, en charger un, le préparer nettoyer etc puis n'enregistrer que cela.

Si toujours pas suffisant, il faudra faire les "réductions de dimensions" (IDF etc) séparemment et trouver un moyen de les assemblées ensuite.

In [ ]:
def get_only_chosen_site(df_all:pd.DataFrame, list_chosen_site) -> pd.DataFrame :
    """
        Desc. : La fonction permet de ne garder que les articles provenant des sites choisis
        
        Paramètre :
            df_all : le dataframe de tous les CVS
            list_chosen_site : la liste des noms des sites choisis
            
        Out :
            le nouveau dataFrame avec uniquement les sites voulus
    
    
    """
    listeOK = []
    for i in df_all.itertuples():
        listeOK.append(i.site in list_chosen_site)

    df_all['test'] = listeOK
    df_all = df_all[df_all['test']]
    df_all = df_all.drop(columns='test')

    return df_all

In [ ]:
df_all = get_only_chosen_site(df_all, list_chosen_site)
#np.unique(df_all['site'])

In [ ]:
def cleanDate(df: pd.DataFrame, date_column: str, taux_nan: float):
    """
    Fonction : 
        - Nettoyage des lignes avec des champs non référencés si < tauxNan.
        - Formatage des dates au formats MM YYYY
    
    
    Paramètres :
        df : le dataFrame issu des CSV Nature
        date_column : la nom de la conlonne contenant les dates
        taux_nan : le taux de lignes contenant des nan en dessous duquel on accepte la suppression des lignes
    
    Sorties :
        N.A. (modification du dataframe fournit en entrée)
    
    """
    # Suppression des lignes avec nan si inf à taux_nan du nb d'article tt + recalcul du nbArticle
    ligneNan = df.index[df.isnull().any(axis=1)]
    nbLigneNan = len(ligneNan)
    nbArct = df.shape[0]
    
    if nbLigneNan != 0:
        if nbLigneNan < taux_nan * nbArct:
            df = df.drop(ligneNan, 0, inplace=True)
            nbArct = df.shape[0]   
    
    
    #Corr° des dates
    corrDate = [('January', '01'),
                ('February', '02'),
                ('March', '03'),
                ('April', '04'),
                ('May', '05'),
                ('June', '06'),
                ('July', '07'),
                ('August', '08'),
                ('September', '09'),
                ('October', '10'),
                ('November', '11'),
                ('December', '12')]

    cleanDate = []
    date = np.array(df[date_column])
    
    for i in date:
        try:
            if type(int(i[0:2])) is int:
                dateTemp = i[3:]

        except:
            dateTemp = i

        for k, v in corrDate:
            if dateTemp.replace(k, v) != dateTemp:
                cleanDate.append(dateTemp.replace(k, v))

                
    if len(cleanDate) != 0:
        df[date_column] = cleanDate

In [ ]:
cleanDate(df_all, 'date', 0.1)
#df_all[0:5]

In [ ]:
def clean_text(text: str) -> str:
    """
    Desc. :
        Cette fonction permet de nettoyer le texte fournit par suppr des stop_word, lemmatization etc.

    Parametre :
        text: issus par exp de df.iloc[0]['text']

    Out :
        Ressort le texte nettoyé

    """

    text = nlp(text)
    text_clean = []

    for token in text:
        if token.is_stop == False and token.pos_ not in ['PUNCT', 'SPACE'] and token.is_alpha == True:
            text_clean.append(token.lemma_)

    return (" ".join(text_clean))

In [ ]:
text = df_all.iloc[2]['text']
print(text[0:200], "\n")
text_nett = clean_text(text)
print(text_nett[0:200])

In [ ]:
#Nettoyage des textes (long)
all_text_clean = []

for i in tqdm(df_all.itertuples()):
    text_clean = clean_text(i.text)
    all_text_clean.append(text_clean)

In [ ]:
#On ecrase les anciens text par les cleans
df_all['text'] = all_text_clean

#df_all[0:5]['text']

Voir avec le scrappe si on ne peut pas être plus précis (récupérer le type, l'auteur etc à part et ne garder que les abstracts des textes par exp) ==> V1